In [102]:
import pandas as pd 
import numpy as np
import requests
import io
import csv
from path import Path

In [73]:
#eventually we will replace this with a database
#url = 'https://github.com/AForbis/Bootcamp_Final_Project/blob/datasourcing/SeasonData/AllNBA_Unpivoted.xlsx'
url= Path("Resources/AllNBA_UNpivoted.xlsx")
#load in All NBA teams, drop the league column (we are not including the ABA, I already deleted all that from the file)
All_NBA_Unpivoted_df = pd.DataFrame(pd.read_excel(url)).drop(['Lg'],axis=1)
#rename the players column
All_NBA_Unpivoted_df=All_NBA_Unpivoted_df.rename(columns = {'Value':'PlayerName'})
#create column with the start year
All_NBA_Unpivoted_df['Season_Start_Year'] = All_NBA_Unpivoted_df['Season'].apply(lambda row: row[0:4])
All_NBA_Unpivoted_df['Season_Start_Year'] = pd.to_numeric(All_NBA_Unpivoted_df['Season_Start_Year'])
#All_NBA_Unpivoted_df.head(10)

In [74]:
#cleaning up the names with crazy characters
All_NBA_Unpivoted_df['PlayerName_Formatted'] = All_NBA_Unpivoted_df['PlayerName'].apply(lambda row: row.replace("ć","c"))
All_NBA_Unpivoted_df['PlayerName_Formatted'] = All_NBA_Unpivoted_df['PlayerName_Formatted'].apply(lambda row: row.replace("č","c"))
All_NBA_Unpivoted_df['PlayerName_Formatted'] = All_NBA_Unpivoted_df['PlayerName_Formatted'].apply(lambda row: row.replace("ž","z"))

#make dummies for position and team (1st, 2nd, 3rd)
All_NBA_Unpivoted_df = pd.get_dummies(All_NBA_Unpivoted_df,columns=['Position','Tm']).drop(['Season','PlayerName'],axis=1)
#rename the columns (cosmetic)
All_NBA_Unpivoted_df=All_NBA_Unpivoted_df.rename(columns = {'Position_C':'AllNBA_Center'})
All_NBA_Unpivoted_df=All_NBA_Unpivoted_df.rename(columns = {'Position_F':'AllNBA_Forward'})
All_NBA_Unpivoted_df=All_NBA_Unpivoted_df.rename(columns = {'Position_G':'AllNBA_Guard'})
#All_NBA_Unpivoted_df.head(10)

In [75]:
#get list of all of the players that have made an all nba team
AllNBAPlayersList = All_NBA_Unpivoted_df['PlayerName_Formatted'].unique()
#AllNBAPlayersList

In [76]:
#url = 'SeasonData/Seasons_Stats.xlsx'
url= Path("Resources/Seasons_Stats.xlsx")
#load in season stats
season_stats_df = pd.DataFrame(pd.read_excel(url))
season_stats_df['Player'] = season_stats_df['Player'].apply(lambda row: row.replace("*",""))
season_stats_df=season_stats_df.rename(columns = {'Year':'Year_season_end'})
season_stats_df['Year_season_start'] = (season_stats_df['Year_season_end'] - 1)
#season_stats_df.head()
#season_stats_df.to_csv('ML_DF.csv')

In [77]:
# get list of every nba player
AllPlayersList = season_stats_df.Player.unique()
AllPlayersList_df = pd.DataFrame(AllPlayersList)
#AllPlayersList_df.to_csv('listofplayers_test.csv')

In [78]:
#check and make sure every all-nba player shows up in the season data dataset
#players who weren't in the league in 2017 will show up here, that's okay!
#I cleaned up a bunch of names in the excel
for player in AllNBAPlayersList:
    if player not in AllPlayersList:   
        print(player)

Luka Doncic
Jayson Tatum
Ben Simmons


In [79]:
#merge allNBA status into season stats
Season_Stats_ML_DF = pd.merge(season_stats_df,All_NBA_Unpivoted_df,how='left',left_on = ['Player','Year_season_start'], right_on = ['PlayerName_Formatted','Season_Start_Year'])

In [80]:
# get rid of duplicate cols
Season_Stats_ML_DF = Season_Stats_ML_DF.drop(['Season_Start_Year','PlayerName_Formatted'],axis=1)

In [81]:
# fill in zeros for players that arent all nba
Season_Stats_ML_DF['AllNBA_Center'] = Season_Stats_ML_DF['AllNBA_Center'].fillna(int(0))
Season_Stats_ML_DF['AllNBA_Forward'] = Season_Stats_ML_DF['AllNBA_Forward'].fillna(int(0))
Season_Stats_ML_DF['AllNBA_Guard'] = Season_Stats_ML_DF['AllNBA_Guard'].fillna(int(0))
Season_Stats_ML_DF['Tm_1st'] = Season_Stats_ML_DF['Tm_1st'].fillna(int(0))
Season_Stats_ML_DF['Tm_2nd'] = Season_Stats_ML_DF['Tm_2nd'].fillna(int(0))
Season_Stats_ML_DF['Tm_3rd'] = Season_Stats_ML_DF['Tm_3rd'].fillna(int(0))


In [82]:
#get list of positions
Positions_List = Season_Stats_ML_DF['Pos'].unique()


In [83]:
# create functions to determine if player is a guard forward and/or center
def is_forward(pos):
    possible_pos = [ 'SF',  'PF', 'PF-C', 'SG-SF', 'PF-SF', 'SF-SG', 'SF-PF', 'C-PF', 'SG-PF', 'C-SF', 'PG-SF']
    if pos in possible_pos: isF = 1
    else: isF = 0
    return isF

def is_guard(pos):
    possible_pos = ['PG','SG','SG-SF','PG-SG','SG-PG','SF-SG','SG-PF','PG-SF']
    if pos in possible_pos: isG = 1
    else: isG = 0
    return isG

def is_center(pos):
    possible_pos = ['C','PF-C','C-PF','C-SF']
    if pos in possible_pos: isC = 1
    else: isC = 0
    return isC

In [84]:
#Create dummy columns for G, F, C
Season_Stats_ML_DF['is_Guard'] = Season_Stats_ML_DF['Pos'].apply(lambda pos: is_guard(pos) )
Season_Stats_ML_DF['is_Forward'] = Season_Stats_ML_DF['Pos'].apply(lambda pos: is_forward(pos) )
Season_Stats_ML_DF['is_Center'] = Season_Stats_ML_DF['Pos'].apply(lambda pos: is_center(pos) )


In [85]:
#Drop the Pos Column since we have dummy columns
Season_Stats_ML_DF=Season_Stats_ML_DF.drop(['Pos'],axis=1)


In [86]:
#make dummies for team
Season_Stats_ML_DF = pd.get_dummies(Season_Stats_ML_DF,columns=['Tm','Year_season_start'])


In [87]:
# drop year end and player name
Season_Stats_ML_DF=Season_Stats_ML_DF.drop(['Year_season_end','Player'],axis=1)
Season_Stats_ML_DF.head()

,Age,G,GS,MP,PER,TS%,3PAr,FTr,ORB%,DRB%,...,Year_season_start_2007,Year_season_start_2008,Year_season_start_2009,Year_season_start_2010,Year_season_start_2011,Year_season_start_2012,Year_season_start_2013,Year_season_start_2014,Year_season_start_2015,Year_season_start_2016
0,24,22,0.0,163,8.4,0.472,0.238,0.476,5.4,20.9,...,0,0,0,0,0,0,0,0,0,1
1,32,65,0.0,894,9.5,0.507,0.427,0.133,2.3,6.3,...,0,0,0,0,0,0,0,0,0,1
2,21,80,72.0,2298,14.4,0.530,0.309,0.251,5.3,14.1,...,0,0,0,0,0,0,0,0,0,1
3,22,5,0.0,17,-2.2,0.102,0.500,0.500,0.0,19.5,...,0,0,0,0,0,0,0,0,0,1
4,25,18,0.0,135,14.4,0.505,0.278,0.352,7.7,21.0,...,0,0,0,0,0,0,0,0,0,1


In [28]:
#Season_Stats_ML_DF.to_csv(r'C:\Users\18323\Desktop\Classwork\Final_Project\export_dataframe.csv', index = False, header=True)

In [88]:
#Creating Per Game Statistics

#Points per Game
Season_Stats_ML_DF["PPG"] = Season_Stats_ML_DF["PTS"]/Season_Stats_ML_DF["G"]
#Assits Per Game
Season_Stats_ML_DF["APG"] = Season_Stats_ML_DF["AST"]/Season_Stats_ML_DF["G"]
#Rebounds Per Game
Season_Stats_ML_DF["RPG"] = Season_Stats_ML_DF["TRB"]/Season_Stats_ML_DF["G"]
#Blocks Per Game
Season_Stats_ML_DF["RPG"] = Season_Stats_ML_DF["BLK"]/Season_Stats_ML_DF["G"]
#STLs Per Game
Season_Stats_ML_DF["SPG"] = Season_Stats_ML_DF["STL"]/Season_Stats_ML_DF["G"]

In [103]:
#machine learning part



#Here is the code for how to "Replace NaN with zero and infinity with large finite numbers." using numpy.nan_to_num.

Season_Stats_ML_DF[:] = np.nan_to_num(Season_Stats_ML_DF)

#X = Season_Stats_ML_DF_loans.copy()
X = Season_Stats_ML_DF.drop("AllNBA_Center", axis=1)
X.head()        
                                                                        

,Age,G,GS,MP,PER,TS%,3PAr,FTr,ORB%,DRB%,...,Year_season_start_2011,Year_season_start_2012,Year_season_start_2013,Year_season_start_2014,Year_season_start_2015,Year_season_start_2016,PPG,APG,RPG,SPG
0,24.0,22.0,0.0,163.0,8.4,0.472,0.238,0.476,5.4,20.9,...,0.0,0.0,0.0,0.0,0.0,1.0,2.181818,0.181818,0.590909,0.045455
1,32.0,65.0,0.0,894.0,9.5,0.507,0.427,0.133,2.3,6.3,...,0.0,0.0,0.0,0.0,0.0,1.0,4.953846,1.923077,0.138462,0.384615
2,21.0,80.0,72.0,2298.0,14.4,0.530,0.309,0.251,5.3,14.1,...,0.0,0.0,0.0,0.0,0.0,1.0,12.737500,1.875000,0.500000,0.800000
3,22.0,5.0,0.0,17.0,-2.2,0.102,0.500,0.500,0.0,19.5,...,0.0,0.0,0.0,0.0,0.0,1.0,0.200000,0.600000,0.000000,0.000000
4,25.0,18.0,0.0,135.0,14.4,0.505,0.278,0.352,7.7,21.0,...,0.0,0.0,0.0,0.0,0.0,1.0,3.500000,0.388889,0.388889,0.444444


In [104]:
y = Season_Stats_ML_DF["AllNBA_Center"].values.reshape(-1, 1)
y[:5]

array([[0.],
       [0.],
       [0.],
       [0.],
       [0.]])

In [105]:
# Splitting into Train and Test sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [106]:
from sklearn.preprocessing import StandardScaler
# Creating StandardScaler instance
scaler = StandardScaler()
# Fitting Standard Scaller
X_scaler = scaler.fit(X_train)
# Scaling data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [108]:
import pandas as pd
from path import Path
from sklearn import tree
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn import tree
# Creating the decision tree classifier instance
model = tree.DecisionTreeClassifier()

model = model.fit(X_train_scaled, y_train)

# Making predictions using the testing data
predictions = model.predict(X_test_scaled)
predictions



# Calculating the confusion matrix
cm = confusion_matrix(y_test, predictions)
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"]
)

# Calculating the accuracy score
acc_score = accuracy_score(y_test, predictions)

In [109]:
# Displaying results
print("Confusion Matrix")
display(cm_df)
print(f"Accuracy Score : {acc_score}")
print("Classification Report")
print(classification_report(y_test, predictions))

Confusion Matrix


,Predicted 0,Predicted 1
Actual 0,4868,12
Actual 1,13,19


Accuracy Score : 0.9949104234527687
Classification Report
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00      4880
         1.0       0.61      0.59      0.60        32

    accuracy                           0.99      4912
   macro avg       0.81      0.80      0.80      4912
weighted avg       0.99      0.99      0.99      4912

